# In this assignment we learn how to derive a neural network emultor based on the cosmopower emulator (Mancini et al. 2021). Based on this emualtor we perform a PCA data compression.

### For this assignment you need the install cosmopower yourself https://github.com/alessiospuriomancini/cosmopower or you just use the cosmopower_NN.py module that is provided. However in the latter case you need to install tensorflow: https://www.tensorflow.org/install

In [4]:
import matplotlib.pyplot as plt
import random
import numpy as np
from numpy.linalg import inv
import tensorflow as tf
from cosmopower import cosmopower_NN

AttributeError: module 'keras._tf_keras.keras' has no attribute '__internal__'

## Preparation: load in the all the 10000 model vectors and corresponding parameters that we use to bulit the emulator.

In [2]:
models = np.load('data_4_assignment2/models.npy') 
parameters = np.load('data_4_assignment2/parameters.npz')
parameters.keys(),models.shape

(KeysView(NpzFile 'data_4_assignment2/parameters.npz' with keys: omega_m, omega_b, As, w),
 (10000, 900))

#### Divide the models and parameters in a training and test set. 

In [3]:
train_sample = np.arange(0,7000) # select which elements you wanna use for training
test_sample = np.arange(7000,10000) # select which elements you wanna use for training

train_params = {}
for name in parameters.keys():
    train_params[name]=list(np.array(parameters[name])[train_sample])
    
test_params = {}
for name in parameters.keys():
    test_params[name]=list(np.array(parameters[name])[test_sample])

train_features = np.load('data_4_assignment2/models.npy')[train_sample]
test_features = np.load('data_4_assignment2/models.npy')[test_sample]

#### Train the emulator. Reasonable hyper parameres are set already, but test out other settings and comment on what you observe regarding accuracy and speed.

In [4]:
#These premodifications are not necessary but might improve the accuracy
#Here you can try to modify the training features in advance 
features_modified=train_features

cp_nn = cosmopower_NN.cosmopower_NN(parameters=list(parameters.keys()), 
                    modes=np.linspace(-1,1,train_features.shape[1]), 
                    n_hidden = [ 4, 5, 6], # This you should modify. In this exmaple we use three layers with 4, 5 and 6 nodes. Is this enough?
                    verbose=True, # useful to understand the different steps in initialisation and training
                    )

device = 'cpu'
with tf.device(device):
    # train
    cp_nn.train(training_parameters=train_params,
                training_features=features_modified,
                filename_saved_model='data_4_assignment2/emulator_test', # the name of the emulator and where to save it
                # cooling schedule
                validation_split=0.1, # The precentage from train sample that is used for the validation
                learning_rates=[1e-2, 1e-3], # the different leanring rates. This need to be adjusted
                batch_sizes=[100, 100], # the number for models that are used to adjust the NN parameters
                gradient_accumulation_steps = [1, 1],
                patience_values = [100, 100], # Number of epoch to wait before decreasing the learning rate if the loss does not improve anymore
                max_epochs = [200,200], # Maxmimal number of epoch before decreasing the learning rate
                )

AttributeError: type object 'cosmopower_NN' has no attribute 'cosmopower_NN'

#### Check the accuracy of the emulator, by computing how many predictions are inside 68%, 95%, 99%.

In [ ]:
emulated_features = cp_nn.predictions_np(test_params)
# if you modified the training features you need to convert back the output back the original data vectors

diff=(emulated_features/test_features-1) # Measure the relative difference between test sample and the emualted models

mean_diff = ... 
percentiles1 = np.percentile(...) 
percentiles2 = ... 
percentiles3 = ...
bins=range(diff.shape[1])

plt.figure(figsize=(12, 3),dpi=100)
plt.fill_between(...,  label = '$99\%$', alpha=0.8)
plt.fill_between(... label = '$95\%$', alpha = 0.7)
plt.fill_between(...  label = '$68\%$', alpha = 1)
plt.plot(bins,mean_diff,'-',color='black')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(frameon=False, fontsize=15, loc='upper right')
plt.ylabel(r'$| m^{\mathrm{emulated}} - m^{\mathrm{true}}|/  m^{\mathrm{true}}$', fontsize=15)
plt.xlabel(r'vector elements',  fontsize=15)
plt.ylim(-0.001,0.001)
plt.show()

##### Next we want perform a Fisher Analysis using the emulator. We want you to compute the covariance of the parameters $\Theta = \{\Omega_\mathrm{m},w\}$, which can be estimated by $C(\Theta) = F^{-1}$, where $$F_{ij}= \left(\frac{\partial m(\Theta)}{\partial \Theta_i}\right)^{T} C^{-1} \left(\frac{\partial m(\Theta)}{\partial \Theta_i}\right)$$
##### For the partial derivatives we use the five point stencil beam given by $$\frac{\partial m(\Theta)}{\partial \Theta_i}\approx \frac{-m(\Theta_i + 2\ \Delta \Theta_i) + 8 \ m(\Theta_i +  \Delta \Theta_i) - 8 \ m(\Theta_i - \Delta \Theta_i) + m(\Theta_i - 2\ \Delta \Theta_i)}{12 \ \Delta \Theta_i }$$

In [ ]:
Om_shift = ... #decide for a reasonable stepsize in Omega_m
w_shift = ... #decide for a reasonable stepsize in w

# This example compute the model for two different Omega_m values.
paramters = {'omega_m':[0.3,0.31,],'w':[-1,-1],'As':[np.mean(test_params['As'])]*2,'omega_b':[np.mean(test_params['omega_b'])]*2}
    
features_4_div = cp_nn.predictions_np(paramters)
# if you modified the training features you need to convert back the output back the original data vectors

derivative_Om = ... # compute derivative
derivative_w = ... # compute derivative



#### So lets compute the Fisher matrix the corresponding covariance matrix of the paramters:

In [ ]:
cov=np.load('data_4_assignment1/covariance.npy') # We make use of the analy
inv_cov = inv(cov)

FoM_11 = ...
FoM_22 = ...
FoM_12 = ...
FoM_best = np.array([[FoM_11,FoM_12],[FoM_12,FoM_22]])

Cov_parameter = ...
Cov_parameter,np.sqrt(np.diag(Cov_parameter))


#### First you need to generate the transformation from our model/data vectors to the PCA elements based on the 10k models. Then you need transform all 100k noisy data vectors from which you should then measure the a cvoavraicen on the PCA elements. Now it should get clear why you had to genereate the 100k multvariate Gaussian random variables. 

In [ ]:
from sklearn.decomposition import PCA

N_pca = 900 # Decide how many PCA eigenvalues you want to use
models = np.load('data_4_assignment2/models.npy')
mean = np.mean(models,axis=0) # for numerical stability you should subtract the mean of each element
pca = PCA(n_components=N_pca,svd_solver='full')
models_pca = pca.fit_transform(models-mean) # Perform the PCA fitting. Now you can use pca for the transformation

#rotate covariance matrix. If you do not how to do that. You can also create 10k random data vectors and tranform each one individual, 
# and the compute the covariance matrix from them
rotation_matrix = pca.components_.T
cov_pca = ...

plt.imshow(cov_pca/np.outer(np.sqrt(np.diag(cov_pca)),np.sqrt(np.diag(cov_pca))),vmin=-1)
plt.colorbar()
plt.show()





#### Increase successively the number of PCA elements from which you compute the Fisher matrix. Convince yourself that if you take all possible PCA elements you converge to same contraining power as for original Fisher analysis. How many PCA elements do you need to have 10% and 1% of the constraining power as the original Fisher analysis?

In [ ]:
paramter_constraints = []
for N_pca in np.arange(1,900):
    
    inv_cov_pca = ... # select the first N_pca elements

    features_4_div_pca = pca.transform(...) # transform  the dverative and select the first N_pca elements


    derivative_Om_pca = ... # compute derivative
    derivative_w_pca = ... # compute derivative

    FoM_11 = ...
    FoM_22 = ...
    FoM_12 = ...
    FoM_best_pca = np.array([[FoM_11,FoM_12],[FoM_12,FoM_22]])

    Cov_parameter_pca = inv(FoM_best_pca)
    paramter_constraints.append(np.sqrt(np.diag(Cov_parameter_pca)))
paramter_constraints = np.array(paramter_constraints)

### Plot the constraints depending on the number of PCA that you have used. How many PCA elements do you need to get 10% and 1% of the maximum constraining power.